# Linear Regression

In [ ]:
case class house(x1: Long, x2: Long, y: Long)

val trainData = spark.createDataFrame(Seq(house(2104, 3, 400), house(1600, 3, 330), house(2400, 3, 369),
                                          house(1416, 2, 232), house(3000, 4, 540))).toDF

val testData = spark.createDataFrame(Seq(house(4000, 4, 0))).toDF

In [ ]:
import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().setInputCols(Array("x1", "x2")).setOutputCol("features")

val train = va.transform(trainData)

val test = va.transform(testData)

In [ ]:
import org.apache.spark.ml.regression.LinearRegression

val lr = new LinearRegression().setFeaturesCol("features").setLabelCol("y").setSolver("normal")

val lrModel = lr.fit(train)

lrModel.transform(test).show

# Generalization, Cross-Validation and Hyperparameter Tuning

In [ ]:
val data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

In [ ]:
import org.apache.spark.ml.regression.LinearRegression

val lr = new LinearRegression().setMaxIter(10)

val lrModel = lr.fit(data)

In [ ]:
println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")

val trainingSummary = lrModel.summary

println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")

In [ ]:
val Array(trainDF, testDF) = data.randomSplit(Array(0.8, 0.2))

In [ ]:
import org.apache.spark.ml.regression.LinearRegression

val lr = new LinearRegression().setElasticNetParam(0.8)

val lrModel = lr.fit(data)

In [ ]:
import org.apache.spark.ml.tuning.ParamGridBuilder

val paramGrid = new ParamGridBuilder()
    .addGrid(lr.regParam, Array(0.1, 0.01))
    .addGrid(lr.elasticNetParam, Array(0.0, 1.0))
    .build()

In [ ]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
import org.apache.spark.ml.tuning.CrossValidator
import org.apache.spark.ml.evaluation.RegressionEvaluator

// num folds = 3 => (2 x 2) x 3 = 12 different models being trained
val cv = new CrossValidator()
    .setEstimator(lr)
    .setEvaluator(new RegressionEvaluator())
    .setEstimatorParamMaps(paramGrid)
    .setNumFolds(3)

val cvModel = cv.fit(trainDF)

print(cvModel.bestModel.explainParams)